In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Pending

In [ ]:
# from scipy.io import loadmat

# link = r"D:\external_data\Experiment4"
# filename = r"\Dataset_PWR_WiFi.mat"
# directory = link + filename

# mat = loadmat(directory)

In [ ]:
# # import experimental dataset 2
# folderpath2 = "D:/external_data/Experiment3/csv_files/exp_2"  # CHANGE THIS IF THE PATH CHANGED
# df_exp2 = import_clean_data('exp2',folderpath2)

# lab

### visualisation

In [ ]:
from data.import_data import import_experimental_data

# 1. Import data
folderpath1 = "E:/external_data/Experiment3/csv_files/exp_1"
df = import_clean_data('exp1',folderpath1)


In [ ]:
from models.train import seperate_dataframes
from data.process_data import DatasetObject, slide_augmentation
from data import process_data


X, y = seperate_dataframes(df_exp1)

datasetobj = DatasetObject()
datasetobj.import_data(X, y)
del X, y

In [ ]:

window_size = 900
slide_size = 200
datasetobj.data_transform(lambda x,y,z : process_data.slide_augmentation(x, y, z,
                                                            window_size=window_size,
                                                            slide_size=slide_size,
                                                            skip_labels=['noactivity']),axis=0)
# datasetobj.data_transform(lambda arr: arr.reshape(-1,window_size,3,30).transpose(0,2,1,3),axis=1, col=0)
datasetobj.data_transform(lambda arr: arr.reshape(-1,window_size,3,30).transpose(0,2,3,1),axis=1, col=0)
datasetobj.shape()

In [ ]:
X,y = datasetobj.data[0,0],datasetobj.data[0,1]


num = 102
channel = 0
X,y = X[num][channel],y[num]

# channel = 0
# X = X[channel]
# X = X.reshape(900,30,3)
# X = X.transpose(2,0,1)[0]

for i in [1,3,5,10,30,60]:
    X_ = X[::3,::i]

    print(X_.shape)
    plt.figure(figsize=(15,20))
    plt.imshow(X_,cmap='jet')
    plt.title(y)

In [ ]:
a = X[:,60:90]
print(a.shape)
a.transpose()

In [ ]:
from data.process_data import breakpoints

def breakpoints(ls):
    dic = {}
    for i in range(len(ls)-1):
        if ls[i+1] != ls[i]:
            string = f"{ls[i]}-{ls[i+1]}"
            dic[string] = i
    return dic

points = breakpoints(y_ls[0].reshape(-1).tolist())
print(points)
print('cutpoint: ',y_ls[0].shape[0]//8)

In [ ]:
def normalise():

In [ ]:
user1 = X_ls[0].transpose()
user1_c = user1.reshape(3,30,-1)
user1_bendfwd_ = user1[:,:6999]
user1_kneel_   = user1[:,6999:6999+6999]

In [ ]:
plt.figure(figsize = (15,50))
p = 3000
plt.imshow(user1_c[1][:,p:p+2000],cmap='jet')

In [ ]:
plt.figure(figsize = (15,50))
p = 3000
sample = user1[:,p:p+2000]
# MinMaxScaler().fit_transform(sample)
plt.imshow(sample,cmap='jet')

In [ ]:
y_ls[0][p:p+6999,:]

In [ ]:
user1_X = X_ls[0]
user1_y = y_ls[0]

# Section
# p = 6999*1

for p in range(0,6999*8,6999):

    sample = user1[p:p+6999,:]



    # Scale along first axis 
    # sample = MinMaxScaler().fit_transform(sample)

    # Transpose 
    sample = sample.transpose()

    # ImShow
    plt.figure(figsize = (25,150))
    plt.imshow(sample,cmap='jet')

In [ ]:
6999*2

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_ls[0].shape

In [ ]:
# Visualise spectrogram
img = img.transpose()
img = img[:,::10]
plt.figure(figsize = (15,50))
plt.imshow(img,cmap='jet')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from data.process_data import stacking
import matplotlib.pyplot as plt



def visual_spectrogram(img,title='spectrogram'):
    
    if len(img.shape) == 3:
        
        img.squeeze()
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(img)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
    plt.figure(figsize = (15,50))
    plt.imshow(img,cmap='jet')
    plt.title(title)
    plt.show()
    
    return 

In [ ]:
import torchvision.models as models
        
#visualize weights for alexnet - first conv layer
alexnet = models.alexnet(pretrained=False)

# Store all conv layer in alexnet
k = []
for m in alexnet.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data.shape)
        k.append(m)
    else:  
        continue


In [ ]:
def visual_single_filter(weight):
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(weight)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
#     plt.figure(figsize = (15,10))
    plt.imshow(weight,cmap='jet')
#     plt.show()
    
    return 

def visualise_filters(layer,figsize=(20,15)):
    
    weights = layer.weight.data

    if weights.shape[0] > weights.shape[1]:
        
        weights = weights.transpose(1,0)
        
    rows = weights.shape[0]
    
    cols = weights.shape[1]
    
    axes = []
    
    fig= plt.figure()

    for n in range(rows):

        for c in range(cols):
            
            axes.append(fig.add_subplot(rows, cols, n*cols+c+1))

            single_filter = weights[n,c]
        
            visual_single_filter(single_filter)
            
#     fig.tight_layout()    
    
    plt.show()     
    
    return 

visualise_filters(example)

# model factory

In [1]:
import time
from time import gmtime, strftime

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

In [3]:
from models.baseline import Lambda

##### extra setting

In [33]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

torch.cuda.device_count()

torch.cuda.set_device(device)

cuda:0


In [ ]:
device = torch.device("cuda:0")
torch.cuda.set_device(device)

In [5]:
np.random.seed(1024)
torch.manual_seed(1024)
# torch.set_deterministic(True)

### 0. Default

In [7]:
# model 
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        pass
    
    def forward(self,X):
        return X

### 1. Section

### 2. calling function

In [17]:
assert False

AssertionError: 

# Test

In [18]:
from data import process_data
from data.import_data import import_clean_data
from data.datasetobj import DatasetObject
# from models.train import * 

### 0. shortcut

### 1. Data

In [1]:
from data.experiment_data1 import prepare_exp_1

train_loader, test_loader = prepare_exp_1([0])

Found 80 files.
input_user10_bendfwd.csv annotation_user10_bendfwd.csv user10
input_user10_kneel.csv annotation_user10_kneel.csv user10
input_user10_lie.csv annotation_user10_lie.csv user10
input_user10_sit.csv annotation_user10_sit.csv user10
input_user10_sitrotate.csv annotation_user10_sitrotate.csv user10
input_user10_stand.csv annotation_user10_stand.csv user10
input_user10_standrotate.csv annotation_user10_standrotate.csv user10
input_user10_walking.csv annotation_user10_walking.csv user10
input_user1_bendfwd.csv annotation_user1_bendfwd.csv user1
input_user1_kneel.csv annotation_user1_kneel.csv user1
input_user1_lie.csv annotation_user1_lie.csv user1
input_user1_sit.csv annotation_user1_sit.csv user1
input_user1_sitrotate.csv annotation_user1_sitrotate.csv user1
input_user1_stand.csv annotation_user1_stand.csv user1
input_user1_standrotate.csv annotation_user1_standrotate.csv user1
input_user1_walking.csv annotation_user1_walking.csv user1
input_user2_bendfwd.csv annotation_user2

In [27]:
assert False

AssertionError: 

### 2. Model

In [ ]:
# from models.baseline import Lambda, Classifier,CNN_module

In [28]:
print(torch.cuda.memory_summary(device=device, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [ ]:
del model ,criterion , optimizer

In [ ]:
torch.cuda.empty_cache()

In [29]:
model = create_model()


In [30]:
def setting(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
    return criterion, optimizer

criterion, optimizer = setting(model)

In [3]:
from models import example

### 3. Training

In [31]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
assert False

### 4. Testing

In [ ]:
def evaluation(model,test_loader):
    model = model.cpu()
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test, y_test
            y_val = model(X_test)
            predicted = torch.max(y_val, 1)[1]
    cls = classification_report(y_test.view(-1), predicted.view(-1), output_dict=True)
    cls = pd.DataFrame(cls)
    print(cls)
    cmtx = confusion_matrix(y_test.view(-1), predicted.view(-1))
    return cmtx,cls

def cmtx_table(cmtx,label_encoder=None):
    if label_encoder != None:
        cmtx = pd.DataFrame(cmtx,
                            index=[f"actual: {i}"for i in label_encoder.categories_[0].tolist()], 
                            columns=[f"predict : {i}"for i in label_encoder.categories_[0].tolist()])
    else: 
        cmtx = pd.DataFrame(cmtx)
    return cmtx

cmtx,cls = evaluation(model,test_loader)
cmtx = cmtx_table(cmtx,label_encoder=None)
cmtx

In [ ]:
train_loss = pd.DataFrame(record['train'],columns=['train_loss'])

### 5. Saving

In [ ]:
from models.train import save_checkpoint, make_directory

In [ ]:
path = make_directory('V2cSimCLR',epoch=500,filepath='./record/')
train_loss.to_csv(path+'_loss.csv')
# cls.to_csv(path+'_report.csv')
# cmtx.to_csv(path+'_cmtx.csv')

In [ ]:
path = make_directory('V2cSimCLR_pretrain',epoch=500)
save_checkpoint(model, optimizer, 500, path)